In [14]:
import pandas as pd
from pathlib import Path

In [15]:
# take state college data, multiply it by outcome measures of college, sum or average by column, and we have a measure for the state

In [16]:
ROOT = Path.cwd()
DATA_DIR = ROOT / "datasets"
IPEDS_DIR = DATA_DIR / "ipeds"
MIGRATION_DIR = IPEDS_DIR / "migration"
OUTCOMES_DIR = IPEDS_DIR / "outcomes"
PROCESSED_DIR = IPEDS_DIR / "processed"

In [ ]:
outcomes = pd.read_csv(OUTCOMES_DIR / "om2018_rv.csv")

# num unique schools
print(len(outcomes["UNITID"].unique()))

# first time, full time 

firstFullOutcomes = outcomes[outcomes["OMCHRT"] == 10]

unitIds = firstFullOutcomes["UNITID"] 

# grad rate sanity check
# print(firstFullOutcomes["OMAWDP4"].mean()) 

gradPercent = pd.concat([firstFullOutcomes["UNITID"], firstFullOutcomes["OMAWDP4"]], axis = 1)
gradPercent

3818
37.294934640522875


,UNITID,OMAWDP4
0,100654,7.0
15,100663,30.0
30,100690,100.0
41,100706,15.0
56,100724,7.0
...,...,...
49954,491640,0.0
49960,491710,8.0
49966,492069,17.0
49976,492801,9.0


In [77]:
# grab state makeups data
migration_csv = PROCESSED_DIR / "2018_crushed_migration.csv"
stateMakeup = pd.read_csv(migration_csv)
stateMakeup

,UNITID,Alabama,Alaska,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,District of Columbia,...,Utah,Vermont,Virginia,Washington,West Virginia,Wisconsin,Wyoming,Unknown,US Total,GRAND TOTAL
0,100654,649.0,0.0,1.0,0.0,20.0,4.0,0.0,0.0,8.0,...,3.0,0.0,9.0,0.0,0.0,5.0,0.0,17.0,1400.0,1405
1,100663,1917.0,1.0,3.0,0.0,8.0,3.0,1.0,2.0,0.0,...,0.0,0.0,5.0,1.0,0.0,3.0,0.0,0.0,2269.0,2299
2,100690,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1
3,100706,913.0,1.0,0.0,4.0,6.0,5.0,1.0,0.0,0.0,...,1.0,1.0,14.0,0.0,0.0,3.0,0.0,8.0,1398.0,1435
4,100724,602.0,0.0,0.0,1.0,18.0,0.0,2.0,0.0,2.0,...,0.0,0.0,1.0,1.0,1.0,9.0,0.0,0.0,1024.0,1038
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5795,493150,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10.0,10
5796,493318,0.0,0.0,0.0,0.0,19.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,19.0,28
5797,493372,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,6
5798,493424,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,20.0,20


In [100]:
# reduce data to fewest rows, so we only evaluate schools with both migration and outcome data
reducedMakeup = stateMakeup[stateMakeup["UNITID"].isin(gradPercent["UNITID"])]
reducedGradRate = gradPercent[gradPercent["UNITID"].isin(stateMakeup["UNITID"])]

# row nums should be equal
print(reducedMakeup.shape)
print(reducedGradRate.shape)

(3561, 55)
(3561, 2)


In [104]:

# scaled = reducedMakeup.iloc[:, 1:].mul(reducedGradRate.iloc[: , 1:], axis = 0)
scaled = reducedMakeup.iloc[:, 1:].apply(pd.to_numeric).values * reducedGradRate.iloc[: , 1:].apply(pd.to_numeric, axis = 1).values
scaled


array([[4.543e+03, 0.000e+00, 7.000e+00, ..., 1.190e+02, 9.800e+03,
        9.835e+03],
       [5.751e+04, 3.000e+01, 9.000e+01, ..., 0.000e+00, 6.807e+04,
        6.897e+04],
       [1.000e+02, 0.000e+00, 0.000e+00, ..., 0.000e+00, 1.000e+02,
        1.000e+02],
       ...,
       [0.000e+00, 0.000e+00, 0.000e+00, ..., 0.000e+00, 7.650e+02,
        7.650e+02],
       [0.000e+00, 0.000e+00, 0.000e+00, ..., 0.000e+00, 1.026e+03,
        1.026e+03],
       [1.050e+02, 0.000e+00, 0.000e+00, ..., 0.000e+00, 9.120e+03,
        9.120e+03]], shape=(3561, 54))